### Import necessary libraries

In [1]:
import pandas as pd
from odf.opendocument import load
from odf import text, teletype
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from datasets import Dataset, DatasetDict
from transformers import DataCollatorForSeq2Seq
from tensorflow import keras
import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback
import evaluate
import tensorflow as tf
from transformers import pipeline

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load data from .odt format

In [2]:
def load_data(filepath):
    raw_data = []
    text_doc = load(filepath)
    all_params = text_doc.getElementsByType(text.P)
    for line in all_params:
        raw_data.append(teletype.extractText(line))
    return raw_data

In [3]:
raw_data = load_data('dataset/data.odt')
while '' in raw_data:
    raw_data.remove('')
raw_data

['Wypowiedzi OG',
 'Przywitanie',
 '- Dzień dobry.',
 '[',
 'dzień dobry',
 'witać',
 ']',
 'Wyjaśnienie powodu wizyty',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego.',
 '[',
 'ja wniosek dowód1 mieć',
 'ja wniosek dowód2 mieć',
 ']',
 '- Czy mogę odebrać dowód?',
 '[',
 'czy już nowy dowód1',
 'czy już nowy dowód2',
 'być mój nowy dowód1',
 'być mój nowy dowód2',
 ']',
 '- Chcę zgłosić utratę dowodu osobistego.',
 '[',
 'ja dowód1 zgubić',
 'ja dowód2 zgubić',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojego dziecka.',
 '[',
 'ja dowód1 moje dziecko chcieć',
 'ja dowód2 moje dziecko chcieć',
 ']',
 '- Chcę złożyć wniosek o wydanie dowodu osobistego dla mojej żony.',
 '[',
 'ja dowód1 moja żona chcieć',
 'ja dowód2 moja żona chcieć',
 ']',
 'Obsługa',
 '- Zmieniłem adres.',
 '[',
 'dom przenieść się',
 'mieszkanie zmiana1',
 'mieszkanie zmiana2',
 'adres1 zmiana1',
 'adres2 zmiana1',
 'adres1 zmiana2',
 'adres2 zmiana2',
 ']',
 '- Upłynął termin ważności dowo

 ### Load model and tokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained("./model/tokenizer")
model = TFAutoModelForSeq2SeqLM.from_pretrained("./model/model.h5")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at ./model/model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


### Translation before training

In [39]:
translator = pipeline('translation', model=model, tokenizer=tokenizer)
translator('Czesc jestem Kacper')

[{'translation_text': "Hi, I'm Kacper."}]

### Adjust data form to our model

In [5]:
def split_data_from_list(raw_data):
    pl_sentence = []
    sentence = []
    i=0
    while i < len(raw_data):
        if raw_data[i+1] == '[':
            value = raw_data[i]
            i += 2
            while i < len(raw_data):
                if raw_data[i] == ']':
                    break
                pl_sentence.append(value[1:])
                sentence.append(raw_data[i])
                i += 1
        i += 1
    return pl_sentence, sentence

In [6]:
s1, s2 = split_data_from_list(raw_data)

In [7]:
data = pd.DataFrame({'pl':s1, 'mig':s2})
print(data[0:10])

                                                  pl                     mig
0                                       Dzień dobry.             dzień dobry
1                                       Dzień dobry.                   witać
2   Chcę złożyć wniosek o wydanie dowodu osobistego.  ja wniosek dowód1 mieć
3   Chcę złożyć wniosek o wydanie dowodu osobistego.  ja wniosek dowód2 mieć
4                            Czy mogę odebrać dowód?     czy już nowy dowód1
5                            Czy mogę odebrać dowód?     czy już nowy dowód2
6                            Czy mogę odebrać dowód?     być mój nowy dowód1
7                            Czy mogę odebrać dowód?     być mój nowy dowód2
8             Chcę zgłosić utratę dowodu osobistego.        ja dowód1 zgubić
9             Chcę zgłosić utratę dowodu osobistego.        ja dowód2 zgubić


### Creating a dataset

In [8]:
raw_dataset_list = []
for i in range(0, len(data)):
    raw_dataset_list.append({'translation' : {'pl' : data['pl'][i], 'mig' : data['mig'][i]}})
    
raw_dataset = Dataset.from_list(raw_dataset_list)
raw_dataset

Dataset({
    features: ['translation'],
    num_rows: 123
})

### Split data into train, validation and test dataset

In [9]:
train_test = raw_dataset.train_test_split(test_size=0.25)

train_test_valid_dataset = DatasetDict({
    'train': train_test['train'],
    'test': train_test['test']})

### Create preprocessing function for our data

In [10]:
max_input_length = 32
max_target_length = 32
source_lang = "pl"
target_lang = "mig"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### Map preprocess function on our dataset

In [11]:
tokenized_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/92 [00:00<?, ? examples/s]/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
batch_size = 8

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
train_dataset = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

test_dataset = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

/opt/conda/lib/python3.11/site-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


### Model preparation

In [13]:
model.summary()

Model: "tf_marian_mt_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFMarianMainLayer)   multiple                  77138944  
                                                                 
 final_logits_bias (BiasLaye  multiple                 63430     
 r)                                                              
                                                                 
Total params: 77,202,374
Trainable params: 77,138,944
Non-trainable params: 63,430
_________________________________________________________________


In [14]:
model.model.encoder.trainable = False
model.model.encoder.get_config()

{'name': 'encoder',
 'trainable': False,
 'dtype': 'float32',
 'config': {'vocab_size': 63430,
  'decoder_vocab_size': 63430,
  'max_position_embeddings': 512,
  'd_model': 512,
  'encoder_ffn_dim': 2048,
  'encoder_layers': 6,
  'encoder_attention_heads': 8,
  'decoder_ffn_dim': 2048,
  'decoder_layers': 6,
  'decoder_attention_heads': 8,
  'dropout': 0.1,
  'attention_dropout': 0.0,
  'activation_dropout': 0.0,
  'activation_function': 'swish',
  'init_std': 0.02,
  'encoder_layerdrop': 0.0,
  'decoder_layerdrop': 0.0,
  'use_cache': True,
  'num_hidden_layers': 6,
  'scale_embedding': True,
  'share_encoder_decoder_embeddings': True,
  'return_dict': True,
  'output_hidden_states': False,
  'output_attentions': False,
  'torchscript': False,
  'torch_dtype': None,
  'use_bfloat16': False,
  'tf_legacy_loss': False,
  'pruned_heads': {},
  'tie_word_embeddings': True,
  'is_encoder_decoder': True,
  'is_decoder': False,
  'cross_attention_hidden_size': None,
  'add_cross_attention': 

In [15]:
model.model.decoder.get_config()

{'name': 'decoder',
 'trainable': True,
 'dtype': 'float32',
 'config': {'vocab_size': 63430,
  'decoder_vocab_size': 63430,
  'max_position_embeddings': 512,
  'd_model': 512,
  'encoder_ffn_dim': 2048,
  'encoder_layers': 6,
  'encoder_attention_heads': 8,
  'decoder_ffn_dim': 2048,
  'decoder_layers': 6,
  'decoder_attention_heads': 8,
  'dropout': 0.1,
  'attention_dropout': 0.0,
  'activation_dropout': 0.0,
  'activation_function': 'swish',
  'init_std': 0.02,
  'encoder_layerdrop': 0.0,
  'decoder_layerdrop': 0.0,
  'use_cache': True,
  'num_hidden_layers': 6,
  'scale_embedding': True,
  'share_encoder_decoder_embeddings': True,
  'return_dict': True,
  'output_hidden_states': False,
  'output_attentions': False,
  'torchscript': False,
  'torch_dtype': None,
  'use_bfloat16': False,
  'tf_legacy_loss': False,
  'pruned_heads': {},
  'tie_word_embeddings': True,
  'is_encoder_decoder': True,
  'is_decoder': False,
  'cross_attention_hidden_size': None,
  'add_cross_attention': F

In [16]:
model.summary()

Model: "tf_marian_mt_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFMarianMainLayer)   multiple                  77138944  
                                                                 
 final_logits_bias (BiasLaye  multiple                 63430     
 r)                                                              
                                                                 
Total params: 77,202,374
Trainable params: 25,486,336
Non-trainable params: 51,716,038
_________________________________________________________________


In [17]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, metrics=["accuracy"])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


### Initialize neptune

In [18]:
run = neptune.init_run(
    project="kacperurban/pl-mig-translation",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3MTRhNjcwNy1iMzc2LTQwNTUtOGRjYy03ODI4OGQzNjkxNTEifQ==",
)

neptune_callback = NeptuneCallback(run=run)

/tmp/ipykernel_67/4068754542.py:1: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/kacperurban/pl-mig-translation/e/PLMIG-4


### Model training

In [19]:
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.fit(train_dataset, epochs=3, callbacks=[neptune_callback, early_stopping_callback])
run.stop()

Epoch 1/3
11/11 [==============================] - 38s 749ms/step - loss: 21.9208 - accuracy: 0.0123
Epoch 2/3
11/11 [==============================] - 8s 735ms/step - loss: 7.7565 - accuracy: 0.0195
Epoch 3/3
11/11 [==============================] - 8s 782ms/step - loss: 5.6309 - accuracy: 0.0225
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
All 3 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/kacperurban/pl-mig-translation/e/PLMIG-4/metadata


### Translation after training

In [33]:
from transformers import pipeline

translator = pipeline('translation', model=model, tokenizer=tokenizer)

In [34]:
translator('Czesc jestem Kacper')

[{'translation_text': 'wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'}]